<a href="https://colab.research.google.com/github/barauna-lo/CAP4213-Deep-Learning/blob/main/CAP4213_ProcesImage_barauna_lo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://raw.githubusercontent.com/barauna-lo/CAP4213-Deep-Learning/main/logoinpe.png>

# CAP 421-3 - Deep Learning 
## Classes 01 and 02
### Image Processing

These notebook was part of Deep-Learning course provided by the [INPE](https://www.gov.br/inpe/pt-br) (Instituto Nacional de Pesquisas Espaciais) on his firt edition for the classes "Basics of Digital Image Processing. Convolution: Padding, Filter Dimension, Stride, Dilation." with Professero Tales.

This code is training for learning images. ;)

for: Luan Orion Baraúna

[https://github.com/barauna-lo](https://github.com/barauna-lo)

In [ ]:
#Importin the image file
raster_filename = 'heliponto_2.tif'
!wget https://github.com/tkorting/remote-sensing-images/raw/master/{raster_filename} 

In [ ]:
#Import cv2 libary 
import cv2

opencv_matrix = cv2.imread(raster_filename, cv2.IMREAD_GRAYSCALE)

print(opencv_matrix.shape)
print(type(opencv_matrix))

In [ ]:
# type(opencv_matrix)

In [ ]:
try:
  import rasteirio as rio   
except:
  #rasteio is not installed by 
  #!pip install rasterio
  import rasterio as rio 

with rio.open(raster_filename) as rio_raster:
  rio_matrix = rio_raster.read(1)

  print(type(rio_raster))
  print(rio_matrix.shape)
  print(type(rio_matrix))

In [ ]:
from osgeo import gdal

gdal_raster = gdal.Open(raster_filename, gdal.GA_ReadOnly)
gdal_band = gdal_raster.GetRasterBand(1)
gdal_matrix = gdal_band.ReadAsArray()

print(type(gdal_raster))
print(gdal_matrix.shape)
print(type(gdal_matrix)) 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 35))

plt.subplot(131)
plt.imshow(opencv_matrix, cmap = 'gray')
plt.title('OpenCV Matrix')

plt.subplot(132)
plt.imshow(rio_matrix, cmap = 'gray')
plt.title('rasterio Matrix')

#testing new color 
plt.subplot(133)
plt.imshow(gdal_matrix, cmap = 'ocean')
plt.title('GDAL Matrix')

plt.show() 

In [ ]:
#Printing a peace of the chart
block_matrix = rio_matrix[20:40, 15:25]

plt.figure()
plt.imshow(block_matrix, cmap = 'gray')#,vmin=0, vmax=70)
plt.colorbar()
plt.show()

print(block_matrix)

# Spatial filtering

https://www.youtube.com/embed/hRtmSh2gF48?start=42

<center><iframe width="800" height="500" src="https://www.youtube.com/embed/hRtmSh2gF48?start=42" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>

# Exercise 1

Create a convolution kernel called `average_kernel` as a `numpy array` with the following parameters:
* width: 15
* height: 15
* weights: all values equal to 1 / (15 * 15) or `~0.00444`

then apply it to `raster_filename`.

Display, using `matplotlib`, the input (`rio_matrix`) and output (`conv_matrix`) images.

In [ ]:
import numpy as np

#Inicial Parametes 
width = 15
height = 15
weigth = 1/(15*15)

color = 'gray'

# Creating a midpoint
width_mean  = int( width/2)
height_mean = int(height/2) 


new_matrix = rio_matrix.copy() #The matrix we use subtitution 
rows   = new_matrix.shape[0]   #lent os rows
coluns = new_matrix.shape[1]   #lent of colluns

#Creating a The Avarege Kernel
avarege_kernel = (np.ones((width,height)))*weigth #creatind a matrix with 1 ones
  
#Creating a Empty Matrix     
conv_matrix=[]
conv_matrix= np.zeros_like(new_matrix)


for i in range(height_mean,rows-height_mean):
    for j in range(width_mean,coluns-width_mean):
        box_matrix = new_matrix[i-height_mean : i + height_mean +1 , 
                                j-width_mean  : j + width_mean  +1]
        conv_matrix[i,j] = np.dot(box_matrix.flatten(),avarege_kernel.flatten())

#Ploting        
plt.figure(figsize=(15,35))
plt.subplot(121)
plt.imshow(rio_matrix, cmap = color)#,vmin=0, vmax=100)
plt.title('Input')

plt.subplot(122)
plt.imshow(conv_matrix, cmap= color)
plt.title('Filter')

plt.show()


# Exercice 1
## No-Padding effect

Here we solve the Pading problem by creating a condition for the size of the box_matrix. Unlike adding columns with zeros or ones, with this logic the kernal will not perform the multiplication with false weights. In this way, only the points that are in the vicinity of the central pixel will be taken into account. If it does not exist, it will not be taken into account.

For that, I defined four functions that will delimit the existence condition and limit the box size.

In [ ]:
import numpy as np

#Inicial Parametes 
width  = 15
height = 15
weigth = 1/(15*15)

color = 'gray'

# Creating a midpoint
width_mean  = int( width/2)
height_mean = int(height/2) 

new_matrix = rio_matrix.copy() #The matrix we use subtitution 
rows   = new_matrix.shape[0]   #lent os rows
coluns = new_matrix.shape[1]   #lent of colluns

#Creating a variable size for the box_matrix
def lower_width(x):  return 0                    if x < width_mean           else x - width_mean
def upper_width(x):  return x + width_mean +1    if x < rows-width_mean      else rows
def lower_height(x): return 0                    if x < height_mean          else x - height_mean
def upper_height(x): return x +height_mean +1    if x < coluns-height_mean   else coluns 

#Creating a Empty Matrix     
conv_matrix=[]
conv_matrix= np.zeros_like(new_matrix)

#Defining 
for i in range(rows):
    for j in range(coluns):
        box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
                                lower_height(j) : upper_height(j)]
        avarege_kernel = np.ones((box_matrix.shape[0],box_matrix.shape[1]))*weigth #creating the kernel 
        conv_matrix[i,j] =  np.dot(box_matrix.flatten(),avarege_kernel.flatten())  #alocating the conv_matrix

#Plotting        
plt.figure(figsize=(15,35))
plt.subplot(121)
plt.imshow(rio_matrix, cmap = color)#,vmin=0, vmax=100)
plt.title('Input ('+str(rio_matrix.shape[0])+','+str(rio_matrix.shape[1])+')')

plt.subplot(122)
plt.imshow(conv_matrix, cmap= color)
plt.title('No Pading ('+str(conv_matrix.shape[0])+','+str(conv_matrix.shape[1])+')')

plt.show()


# Exercise 2
Using the same convolution kernel defined in the previous exercise, apply it to `raster_filename` using a stride of 5.

Display, using `matplotlib` input (`rio_matrix`) and output (`conv_matrix_stride`) images.

In [ ]:
import numpy as np

#Inicial Parametes 

weigth = 1/(15*15)

color = 'gray'

#Defining the Kernel
avarege_kernel = np.ones((15,15))*weigth

kernel = avarege_kernel


width  = kernel.shape[0]
height = kernel.shape[1]

# Creating a midpoint
width_mean  = int( width/2)
height_mean = int(height/2) 

new_matrix = rio_matrix.copy() #The matrix we use subtitution 
rows   = new_matrix.shape[0]   #lent os rows
coluns = new_matrix.shape[1]   #lent of colluns

#Creating a variable size for the box_matrix
def lower_width(x):  return 0                    if x < width_mean           else x - width_mean
def upper_width(x):  return x + width_mean +1    if x < rows-width_mean      else rows
def lower_height(x): return 0                    if x < height_mean          else x - height_mean
def upper_height(x): return x +height_mean +1    if x < coluns-height_mean   else coluns 

#The Stride parameter
stride = 5

ones = np.ones_like(new_matrix)

#Defining 
for i in range(0,rows,stride):
    #new_row.append(0)
    for j in range(0,coluns,stride):
        box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
                                lower_height(j) : upper_height(j)]
        new_kernel_matrix =  kernel[0:box_matrix.shape[0],0:box_matrix.shape[1]]
        ones[i,j] = np.dot (box_matrix.flatten(),new_kernel_matrix.flatten())  #alocating the conv_m
        conv_matrix = ones[ones != 1] if stride != 1 else ones #Conditional for stride = 1

#Find the new dimension         
new_row, new_collum = [],[]
for i in range(0,rows,stride):    new_row.append(0)
for i in range(0,coluns,stride):  new_collum.append(0)
new_row, new_collum = len(new_row),len(new_collum)

#Reshaping for the new dimension
conv_matrix = np.array(conv_matrix).reshape(new_row,new_collum)

#Rename
conv_matrix_stride = conv_matrix
        
#Ploting
plt.figure(figsize=(15,35))
plt.subplot(121)
plt.imshow(rio_matrix, cmap = color)#,vmin=0, vmax=100)
plt.title('Input')

plt.subplot(122)
plt.imshow(conv_matrix_stride, cmap= color)
plt.title('Stride = '+str(stride))

plt.show()

# import time
# start_time = time.time()
# print("--- %s seconds ---" % (time.time() - start_time))

# Exercise 3

Create a convolution kernel called `high_pass_kernel` as a `numpy array` with the following weights:

```
-1 -1 -1 -1 -1
 2  2  2 -1 -1
-1  2  2  2 -1
-1 -1 -1  2  2
-1 -1 -1 -1 -1
```

then apply it to `raster_filename`.

Display, using `matplotlib`, the input (`rio_matrix`) image, the output (`conv_matrix`) image, and the kernel.

In [ ]:
import numpy as np

# Control Parameners
#defining the kernel
high_pass_kernel = np.array([ 
[-1, -1, -1, -1, -1],
[ 2,  2,  2, -1, -1],
[-1,  2,  2,  2, -1],
[-1, -1, -1,  2,  2],
[-1, -1, -1, -1, -1]])

stride = 1

color = 'gray'

kernel = high_pass_kernel

##############################################

#Inicial Parametes 
width  = kernel.shape[0]
height = kernel.shape[1]
#weigth = 1/(15*15)

# Creating a the size of the box
# Creating a midpoint
width_mean  = int( width/2)
height_mean = int(height/2) 

leng = width*height            #lenth of width vector
new_matrix = rio_matrix.copy() #The matrix we use subtitution 
rows   = new_matrix.shape[0]   #lent os rows
coluns = new_matrix.shape[1]   #lent of colluns

#Creating a variable size for the box_matrix
def lower_width(x):  return 0 if x < width_mean else x - width_mean
def upper_width(x):  return x + width_mean +1 if x < rows-width_mean else rows
def lower_height(x): return 0 if x < height_mean else x - height_mean
def upper_height(x): return x +height_mean +1 if x < coluns-height_mean else coluns 
    
ones = np.zeros_like(new_matrix)

#Defining 
for i in range(0,rows,stride):
    #new_row.append(0)
    for j in range(0,coluns,stride):
        box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
                                lower_height(j) : upper_height(j)]
        #avarege_kernel          = np.ones((box_matrix.shape[0],box_matrix.shape[1]))*weigth #creating the kernel 
        new_kernel_matrix =  kernel[0:box_matrix.shape[0],0:box_matrix.shape[1]]
        ones[i,j] =  np.dot(box_matrix.flatten(),new_kernel_matrix.flatten())
        conv_matrix = ones#[ones != 1]

#Find the new dimension         
new_row =[]
for i in range(0,rows,stride):  new_row.append(0)
new_row = len(new_row)

new_collum = []
for i in range(0,coluns,stride):  new_collum.append(0)
new_collum = len(new_collum)

#Reshaping for the new dimension
conv_matrix = np.array(conv_matrix).reshape(new_row,new_collum)


#Ploting
plt.figure(figsize=(15,30))
plt.subplot(131)
plt.imshow(rio_matrix, cmap = color)#,vmin=0, vmax=100)
plt.title('Input')

plt.subplot(132)
plt.imshow(conv_matrix, cmap= color)
plt.title('Output')

plt.subplot(133)
plt.imshow(high_pass_kernel, cmap = color)#,vmin=0, vmax=100)
plt.title('High Pass Kernel')

plt.show()


# Exercise 4

Apply a $3 \times 3$ kernel filled with 1's, with two different operations: the first with dilation, and the second with erosion.

Display, using `matplotlib` input (`rio_matrix`) and output (`dilation_matrix` and `erosion_matrix`) images.

In [ ]:
k = 0
for i in range (0,10,2):
    k = k+i
k

In [ ]:
import numpy as np

# Control Parameners
#defining the kernel
dilatarion_matrix = np.array([ 
[0, 1, 0],
[1, 1, 1],
[0, 1, 0]])

erosion_matrix = np.array([ 
[1, 1, 1],
[0, 0, 0],
[0, 0, 0],
])

one_kernels = np.ones((3,3))

stride = 1

color = 'gray'

kernel = dilatarion_matrix/len(dilatarion_matrix)

##############################################

#Inicial Parametes 
width  = kernel.shape[0]
height = kernel.shape[1]
weigth = 1/(15*15)

# Creating a the size of the box
width_mean  = int( width/2)
height_mean = int(height/2) 

#Matrix for substituition
new_matrix  = rio_matrix.copy() 

leng        = width*height #lenth of width vector
rows        = new_matrix.shape[0] #lent os rows
coluns      = new_matrix.shape[1] #lent of colluns

#Creating a variable size for the box_matrix
def lower_width(x):  return 0 if x < width_mean else x - width_mean
def upper_width(x):  return x + width_mean +1 if x < rows-width_mean else rows
def lower_height(x): return 0 if x < height_mean else x - height_mean
def upper_height(x): return x +height_mean +1 if x < coluns-height_mean else coluns 

#Creating a "empyty matrix"
conv_matrix = []
conv_matrix = np.zeros_like(new_matrix)

#Defining 
for i in range(0,rows,stride):
    for j in range(0,coluns,stride):
        box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
                                lower_height(j) : upper_height(j)]
        new_kernel_matrix =  kernel[0:box_matrix.shape[0],0:box_matrix.shape[1]]
        conv_matrix[i,j] =  np.dot(box_matrix.flatten(),new_kernel_matrix.flatten())

##################

kernel = erosion_matrix/len(erosion_matrix)
#Creating a "empyty matrix"
conv_matrix2 = []
conv_matrix2 = np.zeros_like(new_matrix)
#Second Kernel        
for i in range(0,rows,stride):
    for j in range(0,coluns,stride):
        box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
                                lower_height(j) : upper_height(j)]
        new_kernel_matrix =  kernel[0:box_matrix.shape[0],0:box_matrix.shape[1]]
        conv_matrix2[i,j] =  np.dot(box_matrix.flatten(),new_kernel_matrix.flatten())

#Ploting
plt.figure(figsize=(15,35))
plt.subplot(131)
plt.imshow(rio_matrix, cmap = color)#,vmin=0, vmax=100)
plt.title('Input')

plt.subplot(132)
plt.imshow(conv_matrix, cmap= color)
plt.title('Dilatation')

plt.subplot(133)
plt.imshow(conv_matrix2, cmap= color)
plt.title('Erosion')

plt.show()


In [ ]:
dilatarion_matrix*weigth

# BONUS

Here a will test some filters finded in [wikipedia.org/Kernel_(image_processing)](https://en.wikipedia.org/wiki/Kernel_(image_processing))

In [ ]:
import numpy as np

Identiy = np.array([ 
[0, 0, 0],
[0, 1, 0],
[0, 0, 0]])

Edge_detection1 = np.array([ 
[1, 0, -1],
[0, 0, 0],
[-1, 0, 1],
])

Edge_detection2 = np.array([ 
[0, -1, 0],
[-1, 4, -1],
[0, -1, 0],
])

Edge_detection3 = np.array([ 
[-1, -1, -1],
[-1, 8, -1],
[-1, -1, -1],
])

Sharpen = np.array([ 
[0 , -1, 0],
[-1, 5, -1],
[0 , -1, 0],
])

Box_blur = (np.array([ 
[0 , -1, 0],
[-1, 5, -1],
[0 , -1, 0],
]))/9

Gaussian_blur3x3 = (np.array([ 
[1 , 2, 1],
[2, 4, 2],
[1 , 2, 1],
]))/16

Gaussian_blur5x5 = (np.array([ 
[1,4,6,4,1],
[4,16,24,16,4],
[6,24,36,24,6],
[4,16,24,16,4],
[1,4,5,4,1]]))/256

filters = [
Identiy,
Edge_detection1,
Edge_detection2,
Edge_detection3,
Sharpen,
Box_blur,
Gaussian_blur3x3,
Gaussian_blur5x5,
] 

filters_title = [
'Identiy',
'Edge Detection1',
'Edge Detection2',
'Edge Detection3',
'Sharpen',
'Box Blur',
'Gaussian Blur3x3',
'Gaussian Blur5x5',
] 

In [ ]:
# THE CONV_MARIX FUNCTION
def conv_matrix(stride,kernel):
    kernel = kernel/len(kernel)
    #Inicial Parametes 
    width  = kernel.shape[0]
    height = kernel.shape[1]
    weigth = 1/(15*15)

    # Creating a the size of the box
    width_mean  = int( width/2)
    height_mean = int(height/2) 

    #Matrix for substituition
    new_matrix  = rio_matrix.copy() 

    leng        = width*height #lenth of width vector
    rows        = new_matrix.shape[0] #lent os rows
    coluns      = new_matrix.shape[1] #lent of colluns

    #Creating a variable size for the box_matrix
    def lower_width(x):  return 0 if x < width_mean else x - width_mean
    def upper_width(x):  return x + width_mean +1 if x < rows-width_mean else rows
    def lower_height(x): return 0 if x < height_mean else x - height_mean
    def upper_height(x): return x +height_mean +1 if x < coluns-height_mean else coluns 

    #Creating a "empyty matrix"
    #conv_matrix = []
    imput = np.ones_like(new_matrix)

    #Defining 
    for i in range(0,rows,stride):
        for j in range(0,coluns,stride):
            box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
                                    lower_height(j) : upper_height(j)]
            new_kernel_matrix =  kernel[0:box_matrix.shape[0],0:box_matrix.shape[1]]
            imput[i,j] =  np.dot(box_matrix.flatten(),new_kernel_matrix.flatten())
            conv_matrix = imput#[imput != 1]
    new_row =[]
    for i in range(0,rows,stride):  new_row.append(0)
    new_row = len(new_row)

    new_collum = []
    for i in range(0,coluns,stride):  new_collum.append(0)
    new_collum = len(new_collum)

    #Reshaping for the new dimension
    conv_matrix = np.array(conv_matrix).reshape(new_row,new_collum)
    
    
    
    
    return conv_matrix
    
    


# ones = np.ones_like(new_matrix)

# #Defining 
# for i in range(0,rows,stride):
#     #new_row.append(0)
#     for j in range(0,coluns,stride):
#         box_matrix = new_matrix[lower_width(i)  : upper_width(i) , 
#                                 lower_height(j) : upper_height(j)]
#         avarege_kernel          = np.ones((box_matrix.shape[0],box_matrix.shape[1]))*weigth #creating the kernel 
#         ones[i,j] = np.dot (box_matrix.flatten(),avarege_kernel.flatten())  #alocating the conv_m
#         conv_matrix_stride = ones[ones != 1]

# #Find the new dimension         
# new_row =[]
# for i in range(0,rows,stride):  new_row.append(0)
# new_row = len(new_row)

# new_collum = []
# for i in range(0,coluns,stride):  new_collum.append(0)
# new_collum = len(new_collum)

# #Reshaping for the new dimension
# conv_matrix_stride = np.array(conv_matrix_stride).reshape(new_row,new_collum)


In [ ]:
plt.subplot(4,2,i+1)
plt.imshow(conv_matrix(5,filters[i]), cmap = 'gray')#,vmin=0, vmax=100)
plt.title(str(filters_title[i]))
plt.show()

In [ ]:
#conv_matrix(1,one_kernels)
plt.figure(figsize=(15,35))
for i in range(len(filters)):
    plt.subplot(4,2,i+1)
    plt.imshow(conv_matrix(1,filters[i]), cmap = 'gray')#,vmin=0, vmax=100)
    plt.title(str(filters_title[i]))
plt.show()